In [20]:
import asyncio
import inspect
import json
import os
import re
import urllib.parse

import dotenv
import nest_asyncio
import pandas
import requests
from bs4 import BeautifulSoup
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
from langchain_core.documents import Document
from openai import OpenAI
from playwright.async_api import async_playwright
from pydantic import BaseModel, Field

In [21]:
nest_asyncio.apply()

In [22]:
dotenv.load_dotenv(dotenv.find_dotenv())

True

In [23]:
query = "celulares"
page_number = 1

In [24]:
def build_search_url(term: str, page: int = 1) -> str:
    base = "https://www.falabella.com.co/falabella-co/search"
    query = urllib.parse.urlencode({"Ntt": term, "page": page})
    return f"{base}?{query}"

In [25]:
async def get_urls_search_scrapping(term: str, page_number: int = 1) -> list[str]:
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            print("🌐 Navegando...")
            URL = build_search_url(term, page_number)
            await page.goto(URL, wait_until="networkidle", timeout=15000)

            print(page.url)

            final_url = page.url
            if f"&page={page_number}" not in final_url and page_number > 1:
                parsed = urllib.parse.urlparse(final_url)
                qs = urllib.parse.parse_qs(parsed.query)
                qs["page"] = [str(page_number)]
                final_url = parsed._replace(query=urllib.parse.urlencode(qs, doseq=True)).geturl()

                # 3) re-navegar a la URL corregida
                await page.goto(final_url, wait_until="networkidle", timeout=15000)

            print(page.url)
            await page.wait_for_timeout(3000)

            html = await page.content()
            print(f"✅ HTML: {len(html)} caracteres")

            soup = BeautifulSoup(html, "html.parser")

            # Eliminar scripts y estilos
            for tag in soup(["script", "style"]):
                tag.decompose()

            anchors = soup.find_all("a", href=True)
            URLS_SEARCH = []
            for a in anchors:
                href = a["href"]
                if href.startswith("http"):
                    URLS_SEARCH.append(href)

            return URLS_SEARCH

        finally:
            await browser.close()


URLS_SEARCH = asyncio.run(get_urls_search_scrapping(query, page_number))

🌐 Navegando...
https://www.falabella.com.co/falabella-co/category/cat1660941/Celulares-y-Telefonos?sred=celular
https://www.falabella.com.co/falabella-co/category/cat1660941/Celulares-y-Telefonos?sred=celular
✅ HTML: 2151296 caracteres


In [ ]:
URLS_SEARCH

In [27]:
class RegexPattern(BaseModel):
    pattern: str = Field(description="Patrón de expresión regular para detectar URLS_SEARCH de productos. El patrón será evaluado con re.compile(...).")


def get_pattern(URLS_SEARCH: list[str]):
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    model_prompt = inspect.cleandoc(
        f"""
        Las siguientes URLS fueron obtenidas mediante web scraping de un sitio de e-commerce con una consulta de búsqueda de productos.

        URLS_SEARCH:
        URLS_SEARCH = {URLS_SEARCH}

        Analiza estas URLS_SEARCH e identifica un patrón para detectar cuáles corresponden a páginas de productos. Escribe un patrón de expresión regular pra la función get_url_products que reciba este arreglo de URLS_SEARCH y devuelva solo aquellas que corresponden a productos.

        Requisitos:
        - La función debe usar un patrón regex para identificar URLS_SEARCH de productos
        - Considera estructuras comunes de URLS_SEARCH de e-commerce (ej: /product/, /item/, /p/, IDs de productos, etc.)
        - El patrón debe ser específico para evitar falsos positivos pero flexible para capturar variaciones
        - Prueba mentalmente la lógica de tu patrón contra las URLS_SEARCH proporcionadas
        - Incluye la URL del sitio

        Estructura esperada de la función:
        ```python                           
        import re

        def get_url_products(URLS_SEARCH: list[str]) -> list[str]:
            \"\"\"
            Filtra URLS_SEARCH para devolver solo aquellas que corresponden a páginas de productos.
            
            Args:
                URLS_SEARCH: Lista de URLS_SEARCH obtenidas del scraping del sitio de e-commerce
                
            Returns:
                Lista de URLS_SEARCH que coinciden con patrones de páginas de productos
            \"\"\"
            product_pattern = re.compile(r'tu_patron_aqui')
            return [url for url in URLS_SEARCH if product_pattern.search(url)]                           
        ```
        """
    )

    completion = client.beta.chat.completions.parse(
        model="gpt-4.1",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": model_prompt},
                ],
            }
        ],
        temperature=0,
        response_format=RegexPattern,
    )

    event = completion.choices[0].message.parsed

    return event.pattern

In [28]:
def get_url_products(URLS_SEARCH: list[str], pattern_search_urls: str) -> list[str]:
    product_pattern = re.compile(pattern_search_urls)
    return [url for url in URLS_SEARCH if product_pattern.search(url)]

In [ ]:
URLS_PRODUCTS = []
while len(URLS_PRODUCTS) == 0:
    pattern_search_urls = get_pattern(URLS_SEARCH)
    URLS_PRODUCTS = get_url_products(URLS_SEARCH, pattern_search_urls)
URLS_PRODUCTS

In [ ]:
def get_product_scrapping(url_product: str, timeout: int = 10) -> tuple[str, list[str]]:
    try:
        response = requests.get(url_product, timeout=timeout)
        response.raise_for_status()
        html = response.text
    except requests.Timeout:
        print(f"Timeout alcanzado para: {url_product}")
        return "", []
    except requests.RequestException as e:
        print(f"Error al hacer la solicitud: {e}")
        return "", []

    # Procesar HTML con BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")
    for tag in soup(["script", "style"]):
        tag.decompose()

    # Extraer URLs de imágenes
    product_image_urls = [image["src"] for image in soup.find_all("img", src=True) if image["src"].startswith("http")]

    # Transformar HTML a texto con Html2TextTransformer (requiere Document)
    html2text = Html2TextTransformer()
    docs_transformed = html2text.transform_documents([Document(page_content=html)])
    product_page_content = docs_transformed[0].page_content

    return product_page_content, product_image_urls

In [ ]:
class Product(BaseModel):
    id: str = Field(..., description="Identificador único del producto")
    title: str = Field(..., description="Título o nombre del producto")
    price: float = Field(..., description="Precio del producto en formato numérico")
    image_url: str = Field(..., description="URL directa a la imagen del producto. Extrae una de la lista de imagenes teniendo en cuanta id de producto")
    description: str = Field(..., description="Descripción del producto")

In [ ]:
def get_product_info(product_page_content: str, product_image_urls: list[str], url_product: str) -> Product:
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    print(f"Procesando información URL: {url_product}")

    prompt_text = inspect.cleandoc(
        f"""
        A partir de la siguiente información de producto, extrae los siguientes datos estructurados del producto:
        - id: identificador único. se encuantra en la URL y en la mayoría de veces en la descripción.
        - title: nombre del producto
        - price: precio como número
        - image_url: enlace una imagen del producto. coincidencia por id.
        - description: descripción del producto mencionando características técnicas"

        ## URL: {url_product}
        
        ## Descripción de producto
        {product_page_content}

        ## URLS IMAGENES
        URL_IMAGENES = {product_image_urls}
        """
    )

    completion = client.beta.chat.completions.parse(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_text},
                ],
            }
        ],
        response_format=Product,
    )
    event = completion.choices[0].message.parsed
    return event

In [ ]:
# url_product = 'https://www.falabella.com.co/falabella-co/product/137209541/Celular-Oppo-A40-256GB-Lila/137209542'
# product_page_content, product_image_urls = get_product_scrapping(url_product)
# product_info = get_product_info(product_page_content, product_image_urls, url_product)

In [ ]:
def get_products(URLS_PRODUCTS: list[str]) -> dict:

    results = []

    for url_product in URLS_PRODUCTS[:3]:
        product_page_content, product_image_urls = get_product_scrapping(url_product)
        product_info = get_product_info(product_page_content, product_image_urls, url_product)

        results.append(
            {
                "ID": product_info.id,
                "TITLE": product_info.title,
                "PRICE": product_info.price,
                "IMAGE_URL": product_info.image_url,
                "DESCRIPTION": product_info.description,
            }
        )

    return results

In [ ]:
results = get_products(URLS_PRODUCTS)
results

Procesando información URL: https://www.falabella.com.co/falabella-co/product/140755625/CELULAR-XIAOMI-REDMI-NOTE-14-PRO-4g-8+256GB-AZUL/140755626
Procesando información URL: https://www.falabella.com.co/falabella-co/product/73142852/Celular-Samsung-Galaxy-S24-FE-Negro-+-128GB-8GB-RAM-camara-posterior-50-MP-camara-frontal-10-MP-pantalla-6.7-pulgadas-+-Exyno-2400e/73142852?sponsoredClickData=%257B%2522isXLP%2522%253Atrue%252C%2522pageType%2522%253A%2522category%2522%252C%2522adType%2522%253A%2522products%2522%252C%2522platform%2522%253A%2522desktop%2522%252C%2522regionCode%2522%253A%2522co%2522%252C%2522context%2522%253A%257B%2522userIdentifier%2522%253A%2522MTc0OTAwMzUwMzAwOA%253D%253D%2522%252C%2522adIdentifier%2522%253A%2522v2_9fgu7is3vM-LosysmfuhjGqAFJKPlfRSHyyN1jn60jQyJjVZ6dfKXiO1IVh2_HLSyPapIUkpRSCkvyQ9el5ktxm2P0-BrKJENOjVYbpzN_yqJLGIz1EWMWNy5lQ65KRqGXh9PzVGJc4xSYDtS9r1ba5aWESxEVo_b46iv3W31Vw_y_Qii2x2JmqRCmV0IOeqMnu3EFaV-0u1E5Na8GCNWg%253D%253D_ark19738bacfa1134c303%2522%252C%2522

[{'ID': '140755626',
  'TITLE': 'CELULAR XIAOMI REDMI NOTE 14 PRO 4g 8+256GB AZUL',
  'PRICE': 997900.0,
  'IMAGE_URL': 'https://media.falabella.com/falabellaCO/140755626_01/w=800,h=800,fit=pad',
  'DESCRIPTION': 'El Xiaomi Redmi Note 14 Pro 4G ofrece una experiencia móvil excepcional con características destacadas: pantalla OLED CrystalRes de 6.67 pulgadas con resolución 1.5K (1220 x 2712 píxeles) y tasa de refresco de 120 Hz con protección Corning Gorilla Glass Victus 2; procesador MediaTek Dimensity 7300 Ultra octa core a 2.2 GHz, 8 GB de RAM y 256 GB de almacenamiento interno expandible hasta 1TB; cámara principal de 200 MP, cámara ultra gran angular de 8 MP, cámara macro de 2 MP y cámara frontal de 32 MP para selfies; batería de 5500 mAh con carga rápida de 45W; conectividad 4G LTE, WiFi, Bluetooth 5.4 y Android 14 desbloqueado; diseño elegante y dimensiones de 16.2 cm x 7.44 cm x 0.82 cm; otras características incluyen doble SIM nano, pantalla OLED Full HD+ y velocidad de refresc

In [ ]:
df_results = pandas.DataFrame(results)
df_results

,ID,TITLE,PRICE,IMAGE_URL,DESCRIPTION
0,140755626,CELULAR XIAOMI REDMI NOTE 14 PRO 4g 8+256GB AZUL,997900.0,https://media.falabella.com/falabellaCO/140755...,El Xiaomi Redmi Note 14 Pro 4G ofrece una expe...
1,73142852,Celular Samsung Galaxy S24 FE Negro + 128GB 8G...,1819900.0,https://imagedelivery.net/4fYuQyy-r8_rpBpcY7lH...,El Celular Samsung Galaxy S24 FE modelo SM-S72...
2,140072823,Celular Xiaomi Redmi Note 14 Pro Plus 5G 512 G...,1749900.0,https://media.falabella.com/falabellaCO/140072...,El Celular Xiaomi Redmi Note 14 Pro Plus 5G of...


In [ ]:
output_file = open("scrapping_producto.json", "w", encoding="utf-8")
json.dump(results, output_file, ensure_ascii=False, indent=4)
output_file.close()